Patching category

In [ ]:
from backend.database import get_paper, update_paper


def update(title: str, arxiv_category_tag: str) -> None:
    """Temp function to update the arxiv_category_tag of a paper."""
    paper = get_paper(title=title)
    paper.update_arxiv_category_tag(arxiv_category_tag)
    update_paper(paper)

In [ ]:
update(
    "Robust simulation-based inference in cosmology with Bayesian neural networks",
    "astro-ph",
)  # Issue 83

In [ ]:
# Issue 84
paper = get_paper(title="Rapid coarse-grained simulation of whole cryo-tomograms")
paper.category = "Biophysics"
update_paper(paper)

In [ ]:
# Issue 85
paper = get_paper(title="Quantifying binding kinetics with fluorescence microscopy")
paper.category = "Biophysics"
update_paper(paper)

In [ ]:
update(
    "SimBIG: mock challenge for a forward modeling approach to galaxy clustering",
    "astro-ph",
)  # Issue 86

In [ ]:
update(
    "Neural Importance Sampling for Rapid and Reliable Gravitational-Wave Inference",
    "astro-ph",
)  # Issue 87

In [ ]:
!python main.py --crawl

In [ ]:
from backend.api import query_serp

result = query_serp(term="simulation-based+inference")

In [ ]:
result["formatted_results"]

In [ ]:
# Only remake all .md posts
from scholar.post_maker import remake_all_posts

remake_all_posts()

# Archive


## Query SERP for older papers

Get top-1000 relevant papers = 50 queries

In [ ]:
from scholar.api import query_serp, query_arxiv, query_biorxiv
from scholar.database import insert_paper

# term = '"simulation-based+inference"'
# next_url = None

# term = None
# next_url = "https://serpapi.com/search.json?as_sdt=0%2C5&as_vis=1&as_ylo=2000&engine=google_scholar&hl=en&num=20&q=%22simulation-based%2Binference%22&start=100"

# for _ in range(5):
#     results = query_serp(url=next_url, term=term, historical=True)

#     for result in results["formatted_results"]:
#         # Append extra arxiv data
#         if result["journal"] == "arxiv.org":
#             arxiv_data = query_arxiv(result["arxiv_id"])
#             if arxiv_data is not None:
#                 result.update(arxiv_data)

#         # Append extra biorxiv data
#         if result["journal"] == "biorxiv.org":
#             biorxiv_data = query_biorxiv(result["doi"])
#             if biorxiv_data is not None:
#                 result.update(biorxiv_data)

#         # Insert into database
#         insert_result(result)

#         try:
#             next_url = results["serpapi_pagination"]["next"]
#             term = None
#         except KeyError:
#             break